In [1]:
# default_exp torch_core

# Torch Core

> Core pytorch functins

In [2]:
#hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [4]:
# export
from mrl.imports import *
from mrl.torch_imports import *

In [5]:
# export

USE_CUDA = torch.cuda.is_available()